In [1]:
import pandas as pd
import os
from os import listdir
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
import scipy.stats as stats
import zipfile
from sklearn.preprocessing import MinMaxScaler 
import xgboost as xgb
from datetime import date, timedelta

import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde

import warnings
warnings.filterwarnings("ignore")

In [2]:
# functions

def get_latest_master():
    !kaggle datasets download -d benjaminpo/s-and-p-500-with-dividends-and-splits-daily-updated -p data
    zip_file_path = "data/s-and-p-500-with-dividends-and-splits-daily-updated.zip"
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall("data")

    df = pd.read_csv("archive/sp500_stocks.csv")
    sp = df["Symbol"].unique()
    sp.sort()
    sp = list(sp)

    data = os.listdir("data")
    data.sort()
    data = list(data)

    # create master dataframe from single dfs
    df = pd.DataFrame()
    for i in data:
        if i.replace(".csv", "") in sp:
            df2 = pd.read_csv(f"data/{i}")
            df2["company"] = i.replace(".csv", "")
            df = pd.concat([df, df2], ignore_index=True)

    df.to_csv("master1.csv")



def XGB_train_real():

    # short cleaning
    df_x = df4_training.copy()
    df_x.drop(columns=["company", "sector", "subsector"], inplace=True)
    df_x.set_index("date", inplace=True)
    df_x.sort_index(inplace=True)

    # training with data only until training_end
    xg_df = df_x.copy()

    # X y split
    X = xg_df.drop(columns="price_30d")
    y = xg_df["price_30d"]

    # normalization
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # train model
    xgbr = xgb.XGBRFRegressor()
    xgbr.fit(X_scaled, y)

    return xgbr



def get_stocks():

    """
    Output: df with top ten gainers
    """

    yesterday = date.today() - timedelta(days=1)
    df_ga = df4.copy()
    gains_df = pd.DataFrame(columns=["company", "close", "prediction", "gain_predicted"])
    df_ga.drop(columns=["sector", "subsector"], inplace=True)
    df_ga.set_index("date", inplace=True)    
    df_ga.sort_index(inplace=True)

    xg_df = df_ga.loc[str(yesterday)]
    xg_df.sort_index(inplace=True)

    X = xg_df.drop(columns=["company"])
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    pred_xgb = xgbr.predict(X_scaled)
    test = xg_df[["close", "company"]]
    test["prediction"] = pred_xgb
    test["gain_predicted"] = (test["prediction"] - test["close"]) / test["close"] * 100
    test.sort_values(by="gain_predicted", ascending=False, inplace=True)
    #gain = test.head(10)["gain_real"].mean()

    return test



In [3]:
get_latest_master()

Dataset URL: https://www.kaggle.com/datasets/benjaminpo/s-and-p-500-with-dividends-and-splits-daily-updated
License(s): CC-BY-SA-4.0
s-and-p-500-with-dividends-and-splits-daily-updated.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# script creates dataframe for ML
import create_ML1_df
import create_ML1_df_training

# import data
df4 = pd.read_csv("ml1.csv")
df4_training = pd.read_csv("ml1_training.csv")

In [4]:
xgbr = XGB_train_real()

In [5]:
stocks = get_stocks()

In [8]:
stocks.head(20)

,close,company,prediction,gain_predicted
date,,,,
2024-12-11,686.390015,PH,1591.369995,131.846321
2024-12-11,796.030029,LLY,1708.676514,114.649756
2024-12-11,773.000000,REGN,1630.676147,110.954223
2024-12-11,796.969971,URI,1623.795532,103.746137
2024-12-11,642.450012,AXON,1277.942261,98.916995
2024-12-11,648.840027,KLAC,1278.241821,97.004156
2024-12-11,670.559998,INTU,1311.141235,95.529295
2024-12-11,936.559998,NFLX,1708.676514,82.441757
2024-12-11,994.690002,COST,1714.912109,72.406690
